In [1]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-qp_yog4o/farm-haystack_373c8ee9a1a64315822bb8962056b924
  Resolved https://github.com/deepset-ai/haystack.git to commit 75ef9596780a18b4c5ec83777aeede1aef53be89
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-qp_yog4o/farm-haystack_373c8ee9a1a64315822bb8962056b924


In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")

In [4]:
import os
print(os.getcwd())
os.chdir('./drive/MyDrive/pale-companion-files/')

/content


In [5]:
# Import some files
import pickle
with open('./chapter_fmt_list.pkl','rb') as f:
    chapters = pickle.load(f)

# Temporary
# chapters = chapters[:100]

In [6]:
from haystack import Document
chapter_documents = [Document.from_dict(d) for d in chapters]
len(chapter_documents)

307

In [7]:
from haystack.nodes import PreProcessor

word_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=20,
    progress_bar=True, 
    add_page_number=True
)

In [8]:
docs_word = word_preprocessor.process(chapter_documents)
len(docs_word)

Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

53227

In [9]:
# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs_word)

Writing Documents:   0%|          | 0/53227 [00:00<?, ?it/s]

In [10]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
    embed_title=False
    # could sub the models for the facebook ones with larget embedding dims for testing
)

document_store.update_embeddings(retriever)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Updating Embedding:   0%|          | 0/53227 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/3232 [00:00<?, ? Docs/s]

In [11]:
document_store.save("pale_lfqa_v1_wiki.faiss")

In [12]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query="Tell me something about Avery?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=512)



Query: Tell me something about Avery?

{   'content': 'Avery’s worked out cool stuff because she came at Finding from '
               'a neat direction.”  “Milly Legendre doesn’t get that goblins '
               'are individuals and that’s a huge blind spot,” Lucy said.  '
               '“Blind spot so big I ran into it within a few minutes of '
               'talking to her.”  Musser raised a hand, indicating for them to '
               'stop.  Annoyed, Lucy felt the urge to do the opposite of '
               'stopping, adding, “You going to acknowledge those things?  Or '
               'the fact you have your head so firmly up your own ass that you '
               'sacrificed your own son?  ',
    'name': None}

{   'content': 'Avery grabbed her bag, slinging it over her shoulder, and '
               'looked around.  “Over there, we have Mr. Toad and Bubbles-”  '
               'Toadswallow and Bubbleyum looked over as their names were '
               'said.  “Mr. Toad and

In [13]:
from haystack.nodes import Seq2SeqGenerator


generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")


In [14]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)


In [16]:
pipe.run(query="Tell me something about Avery?", params={"Retriever": {"top_k": 5},"Generator":{"top_k":2}})


{'query': 'Tell me something about Avery?',
 'answers': [<Answer {'answer': "I don't know much about Avery, but I can tell you that she has a very strong sense of humor.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['8422036381141cccd84b0e0ed0b576b7', '5fdcae225f38c9237969a1759d321425', 'f25f463b45b954c120c6e47da0c5758d', '2cd8cacbd91ef361f677dccbf956cc13', '2bb5c373d1119d72411d802d6ace1d58'], 'meta': {'doc_scores': [0.5558700921370862, 0.5546289448470512, 0.5541530572937323, 0.5533897980947874, 0.5525755802942748], 'content': ['Avery’s worked out cool stuff because she came at Finding from a neat direction.”  “Milly Legendre doesn’t get that goblins are individuals and that’s a huge blind spot,” Lucy said.  “Blind spot so big I ran into it within a few minutes of talking to her.”  Musser raised a hand, indicating for them to stop.  Annoyed, Lucy felt the urge to do the opposite of stopping, adding, “Yo